In [20]:
import os
from unicodedata import decomposition
import pandas as pd
import numpy as np
import gzip ## library to read zipped files

from collections import Counter
import re
import string

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import decomposition

from sklearn.model_selection import train_test_split

import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Load Data

In [21]:
qa_df = pd.read_csv('data.csv')
qa_df.head(2)

,Unnamed: 0,questionType,asin,answerTime,unixTime,question,answerType,answer
0,0,yes/no,B00000K4MC,"Nov 4, 2014",1.415088e+09,Will it work with Windows 8?,Y,You should look here: http://steamcommunity.co...
1,1,yes/no,B00000K4MC,"Nov 29, 2014",1.417248e+09,Will it work on Windows 7 ?,Y,Yes


In [22]:
qa_df = qa_df[['question','answer']]
qa_df.head(2)


,question,answer
0,Will it work with Windows 8?,You should look here: http://steamcommunity.co...
1,Will it work on Windows 7 ?,Yes


In [23]:
def regex_clean(txt, regex):
    """Replace any text matching the regex

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove matches
    regex : string 
        A text string of the regex pattern you want to match

    Returns
    -------
    The same txt string with the matches removes
    """

    return " ".join(re.sub(regex, "", txt).split())

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


    

In [24]:
# convert to lowercase
qa_df['question'] = qa_df['question'].apply(str.lower)
qa_df['answer'] = qa_df['answer'].apply(str.lower)
qa_df.head(2)

,question,answer
0,will it work with windows 8?,you should look here: http://steamcommunity.co...
1,will it work on windows 7 ?,yes


In [25]:

regex_to_clean = [r'\s\d+\s',
                  r'(//t.co/.+?)\s',
                  r'(//t.co/.+?)',
                  # remove url links
                  r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])'
                 ]

for reg in regex_to_clean:
    qa_df['question'] = qa_df['question'].apply(regex_clean, regex=reg)
    qa_df['answer'] = qa_df['answer'].apply(regex_clean, regex=reg)
    
    
qa_df['question'] = qa_df['question'].apply(remove_emoji)
qa_df['answer'] = qa_df['answer'].apply(remove_emoji)
qa_df.head(2)

,question,answer
0,will it work with windows 8?,you should look here: after the game was rerel...
1,will it work on windows?,yes


In [26]:
# Function to clean special and html tags
def clean_special(text):

    # remove html tags
    clean = re.sub('<.*?>', ' ', text)

    # remove specail characters 
    clean = re.sub('\W+',' ', clean)
    return clean

qa_df['question']=qa_df['question'].apply(clean_special)
qa_df['answer']=qa_df['answer'].apply(clean_special)

qa_df.head()

,question,answer
0,will it work with windows 8,you should look here after the game was rerele...
1,will it work on windows,yes
2,is there a digital download of this game,you can buy rollercoaster tycoon deluxe which ...
3,will this work on a mac,sorry to tell you terresa it won t play
4,i have a new computer with the latest greatest...,this game plays on my lap top just fine if ur ...


### Stopword removal

In [27]:
stpwrds = nltk.corpus.stopwords.words('english')

In [28]:
punc = '!"#$%&()*+, -./:;<=>?@[\]^_`{|}~”“\''

punc = [x for x in punc]

In [29]:
stpwrds.extend(['rt', '#', 'fav', '', ':', '@', '!', ';', '…','...', '(', ')', 'using', 'via','https',"'",'‘'])
stpwrds.extend(punc)

In [37]:
def prep_data(review):
    """ Cleans up a review!

    Parameters
    ----------
    review : string
        A text string that you want to parse and remove matches

    Returns
    -------
    Cleaned up string - tokenized & stemmed!
    """   
    
    ## Pre Token Cleaning - Stuff that applies to a string.
    
    review = review.lower() 
    review = regex_clean(review, r'\s\d+\s')

    ## Go-Go-Token-Rangers
    review = word_tokenize(review)

    ## Post Token Cleaning - Stuff that applies to a list 
        
    lemma = WordNetLemmatizer()

    review = [lemma.lemmatize(word) for word in review]
    review = [word for word in review if word not in stpwrds]
    
    return review

In [38]:
cleaned_qns = [" ".join(prep_data(x)) for x in qa_df['question']]
cleaned_ans = [" ".join(prep_data(x)) for x in qa_df['answer']]

In [39]:
cleaned_qns[:3]

['work window', 'work window', 'digital download game']